<a href="https://colab.research.google.com/github/UK-07/Math-of-Machine-Learning/blob/main/Vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vectorization

Machine Learning involves a lot of fundamental operations that are repeated over parameters, over examples and across layers. While these operations are implemented with nested loops for illustration, that is a very ineffcient operation and does not scale for any significant real-world application. In practical implementations these operations are implemented as Linear Algebra operations, leveraging libraries that are much more optimized for practical applications. \
\
Here will gradually build up these matrix operations to understand the underlying mathematical operations. We will start with a linear-regression operation on a single variable for a single example and proceed to build a Multi-Layer Perceptron that trains over multiple examples.

# 2-Variable Linear Regression
The most fundamental Machine Learning operation is linear regression, which solves a linear equation. \
\
\begin{align}
\mathbf{z} = w_1 x_1 + w_2 x_2 + b
\end{align}

Here:
* $\vec{w}$ = $[w_1, w_2]$ are the model weights learned over training
* $\vec{x}$ = $[x_1, x_2]$ are the features of the input data for a single example
* $b$ is the _bias_

Without vectorization, these are implemented in the following snippet.

In [25]:
import numpy as np
import time

np.random.seed(42)  # Set seed value for repeatability.
x_bar = np.random.randint(0, 10, 2) # Features of the input example.
w_bar = np.random.randint(0, 10, 2) # Weights of the models, typically trained over multiple examples.
b = np.random.randint(0, 10) # Bias value.

print(f"Input features x: {x_bar}")
print(f"Model weights w: {w_bar}")
print(f"Model Bias b: {b}")

Input features X: [6 3]
Model weights W: [7 4]
Model Bias b: 6


In [28]:
start = time.perf_counter_ns()
# Solving for `z`.
z = 0
for x, w in zip(x_bar, w_bar):
  z += x * w
z += b
end = time.perf_counter_ns()

In [30]:
print(f"Solving for z (for-loop): {z} in time {end-start}ns")

Solvin for z (for-loop): 60 in time 224936ns


The above product can also be represented as a dot product of 2 vectors $\vec{\mathbf{x}}=\begin{bmatrix} x_1 & x_2 \end{bmatrix}$ and $\vec{w}=\begin{bmatrix} w_1 & w_2 \end{bmatrix}$: \
\
\begin{align}
\vec{\mathbf{w}} \cdot \vec{\mathbf{x}} &= \begin{bmatrix} w_1 & w_2 \end{bmatrix} \cdot \begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
\\
&= \sum_{i} w_i x_i \\
&= w_1 x_1 + w_2 x_2
\end{align} \

Notice the arrow on $\vec{\mathbf{x}}$ and $\vec{\mathbf{w}}$ indicating they are vectors. \
\
For a single example, this is dot product between two vectors of size $(2, 1)$, resulting in a scalar value for the dot product. This scales for similarly for vectors of all lengths.\
\
We implement this using the dot-product operation in the `numpy` library.

In [66]:
start = time.perf_counter_ns()
z = np.dot(w_bar, x_bar) + b
end = time.perf_counter_ns()

In [32]:
print(f"Solving for z (dot-product): {z} in time {end-start}ns")

Solving for z (dot-product): 60 in time 172189ns


We can see the difference in runtime is a few nano-seconds and it may seem insignificant. However, at this stage, we're dealing with 2 variables on a single example.

## Multiple Examples
Scaling up this example, when we have multiple examples, there are multiple vectors, one $\vec{\mathbf{x}_{i}}$ for each example $i$. In this case we have our weights and input features setup as follows: \
$\vec{\mathbf{w}} = \begin{bmatrix} w_1 \ldots w_n \end{bmatrix}$ and $\mathbf{X} = \begin{bmatrix} \vec{x^{(1)}} \ldots \vec{x^{(m)}} \end{bmatrix}$ for $m$ examples with $n$ features each. \
\
Notice the capitalization of $\mathbf{X}$ indicating that it is a matrix. More specifically: \

\begin{align}
\mathbf{X} \, =
  \begin{bmatrix}
  {x}_{1}^{(1)} & {x}_{1}^{(2)} & \ldots & {x}_{1}^{(m)} \\
  {x}_{2}^{(1)} & {x}_{2}^{(2)} & \ldots & {x}_{2}^{(m)} \\
  \vdots        & \vdots        & \ddots & \vdots \\
  {x}_{n}^{(1)} & {x}_{n}^{(2)} & \ldots & {x}_{n}^{(m)}
  \end{bmatrix}
\end{align} \

To solve for $z_i$ for all examples $i$, our equation now becomes: \

\begin{align}
\vec{\mathbf{z}} \, &= \, \vec{\mathbf{w}} \cdot \mathbf{X}
\\
&= \, \vec{\mathbf{w}} \cdot \begin{bmatrix} \vec{x^{(1)}} & \ldots & \vec{x^{(m)}} \end{bmatrix}
\\
&=
  \begin{bmatrix}
    \vec{\mathbf{w}} \cdot \vec{x^{(1)}} & \ldots & \vec{\mathbf{w}} \cdot \vec{x^{(m)}}
  \end{bmatrix}
\end{align}

\
Notice the arrow on top of $\vec{\mathbf{z}}$ indicating that it is now a vector $\begin{bmatrix} z^{(1)} & \ldots & z^{(m)} \end{bmatrix}$ containing the result for each example. \

We will now implement this step with nested loops and using a dot product.

In [61]:
# We're now using values from 0-1 rather than integers.
_EXAMPLES = 9
_FEATURES = 7
X = np.random.rand(_EXAMPLES, _FEATURES)
w_bar = np.random.rand(_FEATURES)  # Weight matrix is still a vector of weights corresponding each feature.
b = np.random.rand()

# Set a 3-decimal point precision for cleaner printing.
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

print(f"Input features x: \n{X}")
print(f"Model weights w: \n{w_bar}")
print(f"Model Bias b: {b:,.3f}")

Input features x: 
[[ 0.267  0.120  0.230  0.106  0.822  0.139  0.747]
 [ 0.048  0.949  0.711  0.007  0.764  0.514  0.779]
 [ 0.522  0.641  0.786  0.405  0.003  0.850  0.381]
 [ 0.496  0.008  0.492  0.986  0.278  0.395  0.687]
 [ 0.815  0.417  0.310  0.938  0.677  0.119  0.069]
 [ 0.484  0.220  0.254  0.878  0.090  0.715  0.271]
 [ 0.760  0.722  0.007  0.509  0.139  0.866  0.872]
 [ 0.418  0.444  0.303  0.717  0.359  0.268  0.432]
 [ 0.091  0.747  0.013  0.644  0.183  0.232  0.636]]
Model weights w: 
[ 0.993  0.778  0.226  0.176  0.866  0.444  0.137]
Model Bias b: 0.862


In [62]:
start = time.perf_counter_ns()
# Solving for `z`.
z_bar = np.empty(_EXAMPLES)
for i, x_bar in enumerate(X):
  z = 0
  for x, w in zip(x_bar, w_bar):
    z += x * w
  z += b
  z_bar[i] = z
end = time.perf_counter_ns()

In [63]:
print(f"Solving for z (for-loop): {z_bar} in time {end-start}ns")

Solving for z (for-loop): [ 2.167  2.806  2.561  2.156  2.880  2.158  2.895  2.307  1.999] in time 377484ns


In [70]:
start = time.perf_counter_ns()
Z = np.dot(w_bar, X.T) + b
end = time.perf_counter_ns()

In [71]:
print(f"Solving for z (dot-product): {z_bar} in time {end-start}ns")

Solving for z (dot-product): [ 2.167  2.806  2.561  2.156  2.880  2.158  2.895  2.307  1.999] in time 1342508ns


We're now starting to see the performance difference. While the absolute difference is on the scale of a few milliseconds, that is now a few orders of magnitude higher than the single example scenario. We can modify the values of `_EXAMPLES` and `_FEATURES` to see how execution time for both the operations have different growth curves.

### Broadcasting
An interesting note about the above dot-product operation is the addition of the bias. Note how bias is a scalar but the result of `np.dot(X, w_bar)` is a vector of dimension $m$. How does this operation not fail? \
\
That is because of _broadcasting_, which is a process by which `numpy` makes copies of the variable `b` and replicates it to match the dimension of the vector it is being added to. In this case, since the dimension of the output for `np.dot(X, w_bar)` is $m$, `numpy` will replicate `b` into an $m$-dimensional vector prior to the addition operation. Broadcasting operation can also occur for matrix and vector pairs and more generally for tensors of any dimension. \
\
**Note:** The concept of making copies and replication is for illustration only. In practice, this implementation would be terribly inefficient.

## Multi-Layer Perceptron

We're now ready to look at the vectorization for a Multi-Layer Perceptron. In this scenario, we will have multiple neurons in each layer and each neuron will have it's own weights for the input features. This implies there will be a $n$-dimensional weight vector of each neuron $k$:

\begin{align}
  \mathbf{W} \, =
    \begin{bmatrix}
      w_{1, 1} & \ldots & w_{1, n} \\
      w_{2, 1} & \ldots & w_{2, n} \\
      \vdots & \ddots & \vdots \\
      w_{k, 1} & \ldots & w_{k, n}
    \end{bmatrix}
\end{align}

To solve for $z$ for all the examples across all neurons, we now have:

\begin{align}
  \mathbf{Z}_{(k \, \times \, m)} \,
  &= \, \mathbf{W}_{(k \, \times \, n)} \cdot \mathbf{X}_{(n \, \times \, m)}
  \\
  &= \begin{bmatrix}
      w_{1, 1} & \ldots & w_{1, n} \\
      w_{2, 1} & \ldots & w_{2, n} \\
      \vdots & \ddots & \vdots \\
      w_{k, 1} & \ldots & w_{k, n}
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
      {x}_{1}^{(1)} & {x}_{1}^{(2)} & \ldots & {x}_{1}^{(m)} \\
      {x}_{2}^{(1)} & {x}_{2}^{(2)} & \ldots & {x}_{2}^{(m)} \\
      \vdots        & \vdots        & \ddots & \vdots \\
      {x}_{n}^{(1)} & {x}_{n}^{(2)} & \ldots & {x}_{n}^{(m)}
    \end{bmatrix}
  \\
  \\
  &= \begin{bmatrix}
    \sum_{i}w_{1, i} \cdot x_{i}^{(1)} & \ldots & \sum_{i}w_{1, i} \cdot x_{i}^{(m)} \\
    \sum_{i}w_{2, i} \cdot x_{i}^{(1)} & \ldots & \sum_{i}w_{2, i} \cdot x_{i}^{(m)} \\
    \vdots & \ddots & \vdots \\
    \sum_{i}w_{k, i} \cdot x_{i}^{(1)} & \ldots & \sum_{i}w_{k, i} \cdot x_{i}^{(m)} \\
  \end{bmatrix}
\end{align}

\

**Notice** the typeface for $\mathbf{Z}$ indicates it is now a $(k \times m)$ matrix - One output for each of the $k$ neurons across $m$ examples.
\
Again, we will solve these using nested loops and matrix-multiplication to see the runtime difference.

In [92]:
_EXAMPLES = 9
_FEATURES = 7
_NODES = 5

X = np.random.rand(_EXAMPLES, _FEATURES)
W = np.random.rand(_NODES, _FEATURES)  # *NOTE*: Pay attention to the dimensions. Reason will be clear later.
b_bar = np.random.rand(_NODES)  # Each neuron also has it's own bias value so bias is now a vector.

# Set a 3-decimal point precision for cleaner printing.
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

print(f"Input features x: \n{X}")
print(f"Model weights w: \n{W}")
print(f"Model Bias b: \n{b_bar}")

Input features x: 
[[ 0.082  0.218  0.250  0.832  0.463  0.487  0.935]
 [ 0.637  0.899  0.457  0.149  0.930  0.459  0.182]
 [ 0.302  0.051  0.468  0.853  0.934  0.933  0.125]
 [ 0.382  0.028  0.855  0.359  0.631  0.715  0.430]
 [ 0.306  0.962  0.964  0.289  0.480  0.973  0.005]
 [ 0.413  0.100  0.509  0.025  0.407  0.221  0.533]
 [ 0.996  0.801  0.846  0.635  0.042  0.207  0.925]
 [ 0.366  0.104  0.702  0.299  0.269  0.998  0.858]
 [ 0.408  0.980  0.650  0.742  0.032  0.122  0.633]]
Model weights w: 
[[ 0.899  0.624  0.134  0.116  0.844  0.139  0.520]
 [ 0.037  0.081  0.672  0.689  0.341  0.216  0.814]
 [ 0.217  0.542  0.968  0.148  0.941  0.183  0.567]
 [ 0.377  0.965  0.274  0.400  0.983  0.536  0.503]
 [ 0.258  0.701  0.014  0.050  0.599  0.342  0.953]]
Model Bias b: 
[ 0.903  0.714  0.818  0.667  0.126]


In [93]:
start = time.perf_counter_ns()
# Solving for `z`.
Z = np.empty((_NODES, _EXAMPLES))

for i, w_bar in enumerate(W):  # Loop over the 1st axis of W, which nodes.
  for j, x_bar in enumerate(X):  # Loop over the 1st axis of X which is examples.
    z = 0
    for x, w in zip(x_bar, w_bar):
      z += x * w
      z += b[i]  # Bias is a node level entity.
    Z[i][j] = z
end = time.perf_counter_ns()

<ipython-input-93-87f9da57af82>:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  Z[i][j] = z


In [94]:
print(f"Solving for z (for-loop): {Z} in time {end-start}ns")

Solving for z (for-loop): [[ 4.669  5.540  4.833  4.758  4.966  4.541  5.512  4.720  4.909]
 [ 1.889  1.173  1.642  1.661  1.417  1.106  2.022  1.808  1.697]
 [ 8.004  8.599  8.241  8.396  8.645  7.814  8.613  8.230  8.218]
 [ 8.661  9.376  8.946  8.601  9.252  8.021  9.083  8.613  8.821]
 [ 3.110  3.252  2.716  2.737  2.963  2.569  3.395  3.068  3.059]] in time 1763813ns


In [95]:
# Numpy vectors of shape (X,) act wierd with broadcasting operations.
# We delibrately set the shape of the vector to (X, 1) to indicate its a
# column vector.
print(f"Before reshape: {b_bar.shape}")
b_bar = b_bar.reshape((_NODES, 1))
print(f"After reshape: {b_bar.shape}")

Before reshape: (5,)
After reshape: (5, 1)


In [96]:
start = time.perf_counter_ns()
b_bar.reshape(_NODES, -1)
Z = np.dot(W, X.T) + b_bar
end = time.perf_counter_ns()

In [97]:
print(f"Solving for z (dot-product): {z_bar} in time {end-start}ns")

Solving for z (dot-product): [ 2.167  2.806  2.561  2.156  2.880  2.158  2.895  2.307  1.999] in time 1399143ns


Now, the difference is starting to widen. As the dimensions of our data and model increase, so will this difference.